# MIBCI-QCNNs: Implementation

This notebooks automates the simulation and synthesis of the EEGNet-based model HLS design. This design is based on the files with the format `MIBCI-QCNN-X-template.txt` saved in the root directory of the repo. These are the parametriced templates that can be readed from Python and the writed into a `.cpp` (or `.h` it its the header file), forming the followinf files:

- `MIBCI-QCNN.cpp`
- `MIBCI-QCNN.h`
- `MIBCI-QCNN-tb.cpp`

The first one, `MIBCI-QCNN.cpp`  is the file with the algorithmic description of the hardware to systhentize with HLS and then to implement in the FPGA. After the declaration of the activation functions, LeakyReLU and Softmax, there is the top function, MIBCI_QCNN in which the C++ impementation of the model is contained.

The header file, `MIBCI-QCNN.h`, contains the parameters of the dataset (time window, dowsampling, number of classes, number of channels, sampling frecuency...), the architecure paramenters (kernels' sizes, number of filters, LReLU alphas...) and the fixed-point datatype description, which in this case is <16,8>, since it showed the best balance in the number of bits and the accuracy drop.

Finally, the test bench `MIBCI-QCNN-tb.cpp`. It is created to load both the model parameters and the input samples from `.npy` files and validate the model implemented in C++ for the entire dataset. As outputs, it writes a `y_hls_16_8.txt` for each fold, containig the model's output of the validation set of that fold.

## Prepare the model to be implemented

Before running the HLS simualtion, the folder of each model's fold must have an `npyparams` folder containing its pararmeters and another folder called `valdiationDS` with the validation set of the fold, with the inputs (`X.npy`) explited per index, `X_i-npy`. All of this is automated in the [`createnpys.py` `utils` file](utils/createnpys.py).

In [1]:
from utils.createnpys import createnpys

In this case we are going to validate and synthetize the global model trianed with the [`training.ipynb`](training.ipynb) notebook, that its stored in the `global_model` folder.

> It needs to load the model to read its paramters. If you have another process using the GPU (if you have one) it will probably raise an error saying CUDA run out of memory.

In [3]:
createnpys('global_model/')

PROCESSING FOLD 1/5
0.6689342403628118
PROCESSING FOLD 2/5
0.6587301587301587
PROCESSING FOLD 3/5
0.6901927437641724
PROCESSING FOLD 4/5
0.6252834467120182
PROCESSING FOLD 5/5
0.6040249433106576


## HLS C simulation of the model

Since fixed-point datatypes are used to represent the model inputs, interal values, feature maps, paramters and outputs, it is expected to have an accuracy drop between the model implemented in Keras and the its HLS version. To check the output of all the dataset must be acquired using HLS, carefully chosing the model for each sample, since we only want to acquired the valdiation accuracy.

This process is launched using the `launch_csim` function in the [`utils/hls_tools.py`](utils/hls_tools.py) file. It writes the `MIBCI-QCNN.cpp`, `MIBCI-QCNN.h`, and `MIBCI-QCNN-tb.cpp` files with the selected dataset parameters and also the `csim-launcher-template.txt` that is the TCL file that sets up the HLS project, selecting the `xc7z010-clg400-1` part, adding the source and testbench files and luanching the HLS C simulation.

<div class="alert alert-block alert-warning">
<b>Warning!</b> This function must be called from a Vivado HLS-enabled bash, which can be activated using the command:
<code>
source /path/to/Vivado/installation/settings64.sh
</code>
To use this function inside of a Jupyter notebook like this one, just run this command before launching the Jupyter server.
</div>



<div class="alert alert-block alert-warning">
<b>Warning!</b> To reduce the simulation time 5 process are launched in background, one for each fold. This is done with the <code>screen</code> command of Ubuntu. If you aren't on a Linux computer or you don't have at least 5 kernels, you have to use the <code>launch_csim_noparallel</code> function. Just comment the next cell and discomment the following one.
</div>

<div class="alert alert-block alert-warning">
<b>Warning!</b> The entire path mas be passed to the function, since its embebed in the C testbench.
</div>

In [13]:
from utils.hls_tools import launch_csim

launch_csim('/home/daniel/BCI/XOH21/global_model/', T=3, ds=2, Nchans=64, Nclasses=4)

In [10]:
#from utils.hls_tools import launch_csim_noparallel

#launch_csim_noparallel('/home/daniel/BCI/XOH21/global_model/', T=3, ds=2, Nchans=64, Nclasses=4)